- Einleitung in den Datensatz
    - Allgemein Challange
    - Datenstruktur (Felder beschreiben)
    - Zielsetzung

- Visualisierung / Explorative untersuchung
    - Wie viel von was...
    - Cooccurrenten analyse

- Predictig
    - Preprocessing
        - NaN Felder füllen
        - Dummy Felder füllen
    - LinearRegression
    - RandomForest
    - SupportVectorRegression
    - Vergleich unserer Lösung
- Fazit

# Visualisierung / Explorative untersuchung

## Kookkurrents-Analyse der Worte

Die Artisten wurden von jedem Benutzer mit mehreren von 82 gegebenen Wörtern oder dem Tag "None of these" bewertet.
Wenn man nun die Tags pro Artist als "Satz" interpretiert, kann man aus den Tags eine Kookkurrenzliste ableiten.

Die Kookkurrenzliste is ein verschachtelter Hash, bzw. ein `dict`. Der `key` vom Eltern `dict` ist ein Wort. Der `key` vom Kinds `dict` ist das kookkurrierende Wort. Dieses wiederum hat als `value` die frequenz, wie oft es mit dem Eltern-Wort vorkommt.

__Hier der Beginn der von uns erstellten Liste : __

    {
        'Aggressive': {
            'Annoying': 337,
            'Approachable': 145,
            'Arrogant': 1539,
            'Authentic': 515,
            ...
         },
         'Annoying': {
            'Aggressive': 337,
            'Approachable': 18,
            'Arrogant': 372,
            'Authentic': 50,
            ...
         },
         ...
    }

### Berechnen der Signifikanz

Nun kann durch die häufigkeit des gemeinsamem Auftretens die Signifikanz eines Wortes zu einem anderen berechnet werden.
In einfachster form ist dies einfach deren gemeinsames auftreten wie schon erfasst.
Nun gibt es jedoch noch komplexere formen der Signifikanzberechnungen wie der DICE-Koeffizient, die Loglikelihood und der Poisson-Koeffizient. Diese sind alle gerichtete Signifikanzen. Das heißt Wort A ist zu wort B gleich signifikant wie Wort B zu Wort A.

_visualisierung des Graphen_

### Berechnung der Worte mit der höchsten Signifikanz mittels PageRank

Wenn man die Kookkurrenten und deren Signifikanz analog zu Webseiten, welche durch Links auf einander Zeigen, interpretiert. Kann angenommen werden, dass auf die Kookkurrenzen eben so den PageRank-Algorithmus angewendet werden kann. Durch den PageRank-Algorithmus kann herausgefunden werden, welche Kookkurrenten, also Worte am meisten mit anderen Worten zusammen auftreten und so zusagen einen höheren Stellenwert besitzen.

Um den PageRank zu berechnen verwenden wir die Python bibliothek "networkx". Diese erlaubt es einen Graphen auf zu bauen, und bietet die berechnung von PageRank sowie HITS an. Es wurde für jede Signifikanz berechnungsart (Poisson, Loglikelihood und Dice), einen Graphen erstellt und die PageRanks berechnet. Um ein möglichst ausgeglichenes Ergebnis zu erhalten wurden die Ergebnisse zusammengefügt und gemittelt.

__ Die 10 signifikantesten Kookkurrenten __

    [('Distinctive', 0.020537092588989742),
     ('Talented', 0.01961324733238577),
     ('Original', 0.017716415716561188),
     ('Cool', 0.017488920129503163),
     ('Free', 0.017464456020722725),
     ('Authentic', 0.017404108457903064),
     ('Good Lyrics', 0.017352995156741942),
     ('Catchy', 0.017161942730958772),
     ('Current', 0.01652955522327245),
     ('Beautiful', 0.016522639447826164)]

Interessant ist hier, dass ein grossteil der signifikantesten Kookkurrenten, ebenfalls vom RandomForestRegressor als "important Features" angegeben wurde.